In [1]:
#Importing necessary datasets
import xarray as xr
import matplotlib.pyplot as plt 
import numpy as np 


In [4]:
#Manually installing seaborn due to problems with the terminal
%pip install seaborn
#pip show seaborn
import seaborn as sns

Note: you may need to restart the kernel to use updated packages.


In [ ]:
#Importing the datasets and selecting different timesteps to reduce dataset
file = f'/lustre/storeA/project/fou/hi/oper/norkyst_v3/forecast/his/2025/06/01/norkyst800_his_sdepth_20250601T00Z_m00_AN.nc'
ds_norkyst = xr.open_dataset(file, engine="netcdf4").isel(ocean_time = 1)

In [ ]:
#check datafile 
ds_norkyst.head()

Explanation of code:
$\newline$
The transformation is an approach following xroms example on xarrays website:
$\newline$
$Z_0 = \frac{h_c \cdot S + h \cdot C}{h_c + h }$
$\newline$
$z = Z_0 (\zeta + h) + \zeta$
$\newline$
This creates a new datacoordinate Z_rho. 

Explanation: 
The code calculates the midpoint of each s-layer depending on which transformation is found in the V-transform. V-transform == 2 is more regulary used for newer ROMS models according to xroms. 
$\newline$
The calculations creates a variable $Z_0$ which is calculated by the use of:
$\newline$
hc = critical depth for stretching 
$\newline$
$s_rho$ = s-layers 
$\newline$
h = total depth
$\newline$
$Cs_r$ = stretching curve for the rho points
$\newline$
$\zeta$ = free surface  
$\newline$

$Z_0$ explains the depth at a given grid cell using the stretching features including the topography, but it excludes the surface layer.
$z_{rho}$ then includes the surface elevation by including $\zeta$ in the calculations and providing the dataset with a new coordinate defining the depth of the grid cells. 

$\newline$
$\newline$

Finding the 2m values:
1. Take the absolute value of the depths (using z_rho) and then subtracting two, to then be able to pick out the resulting numbers closest to zero. Those closest to zero will be the ones near 2meters depth in the model.
2. The diff.fillna function allows us to have zeros as a value for the landvalues rather than nan, as nan gives errors. Though this is corrected later in the code when plotting with seaborn to ensure the land is visibly different from the rest of the plot.  
3. Using argmin, I am selecting out the index of the number closest to zero, to find which s_layer it belongs to. This is why the dimension being called on is s_rho. 
4. Then selecting the index array defined above of those closest to two meters, I use this as an isel, only selecting out these z_rho values to create a new array. 

In [9]:
def finding_2m_layer(ds_name,index,close_to_2m_val):
    #Define necessary variables used for the transformation from s_layer to depth
    hc = ds_name["hc"] #Critical depth for stretching
    cs_r = ds_name["Cs_r"] #stretching curve at rho points
    zeta = ds_name["zeta"] #.fillna(0) #free-surface 
    H = ds_name["h"] #bathymetry at rho-points (depth)
    theta_s = ds_name["theta_s"] #stretching surface
    theta_b = ds_name["theta_b"] #stretching bottom 
    Vtransform = ds_name["Vtransform"]
    s_rho = ds_name["s_rho"] #range 1,40. 40 is surface layer

    #Transformation process
    if Vtransform == 1:
        Z_0_rho = hc * (s_rho - cs_r) + cs_r * H
        z_rho = Z_0_rho + zeta * (1+Z_0_rho/H)
    elif Vtransform == 2:
        Z_0_rho = (hc * s_rho + cs_r * H) / (hc + H)
        z_rho = zeta + (zeta + H) * Z_0_rho

    ds_name.coords["z_rho"] = z_rho.transpose() #Corrects the dimensions

    #calculating the 2meters depth and creating two new arrays. One controlrun
    #for the actual values and one array containing the indexes of the values.
    z_rho_pos = abs(ds_name["z_rho"])
    diff = abs(z_rho_pos-2)
    diff = diff.fillna(0)
    index = diff.argmin(dim = "s_rho")
    close_to_2m_val = z_rho.isel(s_rho=index)
    #zero = nan in plot
    index = index.where(index != 0)
    ax = sns.heatmap(np.array(index)[::-1,::-1], vmin=25)
    return ds_name, index, close_to_2m_val
